# Higher-order ANOVA Models


## The Higher-order ANOVA Framework

### Terminology and Mean Tables

### Cell Means and Marginal Means


```{admonition} Higher-order terminology
:class: tip
... For instance, the most basic higher-order ANOVA is the two-way ANOVA, that contains two factors. IF each factor had two levels, we would write this as a $2 \times 2$ ANOVA. If the second factor had three levels, we would call it a $2 \times 3$ ANOVA. If there were *three* factors, we would have a three-way ANOVA. If each factor had two levels we could call it a $2 \times 2 \times 2$ ANOVA, and so on.
```

## Adding Another Factor - The Additive Model
Let us start with the most simple approach, which is to simply add an additional factor to the model. In terms of notation, this is a simple extension of what we already known

$$
y_{ijk} = \mu + \alpha_{i} + \beta_{j} + \epsilon_{ijk},
$$

where $\alpha_{i}$ is the effect associated with Factor A and $\beta_{j}$ is the effect associated with Factor B. The simplest form of this model would be one that represents a $2 \times 2$ design, with $i = 1,2$ and $j = 1,2$.

As an example, let us expand our `mtcars` example with an addition categorical predictor. Within `mtcars` there already exists a factor called `vs` which indicates whether the engine is V-shaped or straight. This is already coded as a dummy variable, but we will label it so that it is clearer what it means before turning it into a factor.

In [11]:
data(mtcars)
mtcars$origin <- c('Other','Other','USA','USA','USA','USA','USA','Other','Other','Other',
                   'Other','Other','Other','Other','USA','USA','USA','Other','Other',
                   'Other','Other','USA','USA','USA','USA','Other','Other','Other',
                   'USA','Other','Other','Other')
mtcars$origin <- as.factor(mtcars$origin)

In [12]:
vs.lab <- rep("",length(mtcars$mpg)) 
vs.lab[mtcars$vs == 0] <- "v-shaped"
vs.lab[mtcars$vs == 1] <- "straight"

mtcars$vs <- vs.lab
mtcars$vs <- as.factor(mtcars$vs)
print(levels(mtcars$vs))

[1] "straight" "v-shaped"


We can now see how both `vs` and `origin` are coded.

In [18]:
print(contrasts(mtcars$origin))
print(contrasts(mtcars$vs))

      USA
Other   0
USA     1
         v-shaped
straight        0
v-shaped        1


$$
\begin{align*}
    \mu_{\text{Other,Straight}} &= \beta_{0} + (\beta_{1} \times \mathbf{0}) + (\beta_{2} \times \mathbf{0}) = \beta_{0} \\
    \mu_{\text{USA,Straight}}   &= \beta_{0} + (\beta_{1} \times \mathbf{1}) + (\beta_{2} \times \mathbf{0}) = \beta_{0} + \beta_{1} \\
    \mu_{\text{Other,V-shaped}} &= \beta_{0} + (\beta_{1} \times \mathbf{0}) + (\beta_{2} \times \mathbf{1}) = \beta_{0} + \beta_{2} \\
    \mu_{\text{USA,V-shaped}}   &= \beta_{0} + (\beta_{1} \times \mathbf{1}) + (\beta_{2} \times \mathbf{1}) = \beta_{0} + \beta_{1} + \beta_{2} \\
\end{align*}
$$

Based on this we can see

| Parameter   | Interpretation                   |
|-------------|--------------------------------- |
| $\beta_{0}$ | Mean of `(Other,Straight)` cell   |
| $\beta_{1}$ | Mean difference `(USA,Straight) - (Other,Straight)` |
| $\beta_{2}$ | Mean difference `(Other,V-shaped) - (Other,Straight)`   |

In [28]:
add.mod <- lm(mpg ~ origin + vs, data=mtcars)
print(summary(add.mod))


Call:
lm(formula = mpg ~ origin + vs, data = mtcars)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.7035 -3.2079  0.1795  1.9298  8.3965 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   25.504      1.157  22.036  < 2e-16 ***
originUSA     -4.416      1.587  -2.783  0.00939 ** 
vsv-shaped    -6.433      1.571  -4.094  0.00031 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.139 on 29 degrees of freedom
Multiple R-squared:  0.5588,	Adjusted R-squared:  0.5283 
F-statistic: 18.36 on 2 and 29 DF,  p-value: 7.044e-06



In [22]:
library(effects)
print(allEffects(lm(mpg ~ origin*vs, data=mtcars)))

 model: mpg ~ origin * vs

 origin*vs effect
       vs
origin  straight v-shaped
  Other 25.59091    18.95
  USA   20.76667    14.75


In [24]:
mean(mtcars$mpg[mtcars$origin == 'Other' & mtcars$vs == 'straight'])

[1] 25.59091

### Main Effects

Notice in this model that there is a constant difference between the rows and a constant difference between the columns

$$
SS_{\text{model}} = SS_{A} + SS_{B}
$$

In [ ]:
beta       <- coef(add.mod)
mu.Other_S <- beta[1]
mu.USA_S   <- beta[1] + beta[2]
mu.Other_V <- beta[1] + beta[3]
mu.USA_V   <- beta[1] + beta[2] + beta[3]

add.cell.means <- data.frame("straight"=c(mu.Other_S,mu.USA_S), 
                             "v.shaped"=c(mu.Other_V,mu.USA_V), 
                             row.names=c("Other","USA"))
print(add.cell.means)

      straight v.shaped
Other 25.50350 19.07019
USA   21.08716 14.65385


This also explains why it is possible to get the final cell mean by using $\beta_{0} + \beta_{1} + \beta_{2}$, because this starts at the baseline cell, moves down a row (using the row difference $\beta_{1}$) and then across a column (using the column difference $\beta_{2}$).

However, notice that these estimated means are *not* the actual cell means. If all the effects in the data truly were *additive* then the estimated means would be the same as the cell means. However, because they are not this suggests that something else is going on here. The additive assumption has placed a *constraint* on the model, producing adjusted cell means that allow this assumption to work. Because these differ from the actual cells, the model will not fit as well as a model that lifts this constraint. As we saw above, the additive model assumes that there is a *constant* row difference and a *constant* column difference. But what if that was not true? What is the row difference actually depends upon the column, or if the column difference actually depends upon the row? The fact that the additive model had to use cell means that are different to the true means suggests that this is in fact what is going on. This effect is known as an *interaction*. 

## Adding an Interaction Term - The Full Factorial Model

So, the interaction terms represents that "bit extra" that we need in order to allow the fitted means to be the same as the sample means. The additive model only gets us so far. The interaction terms allows for a cell-specific adjustment to shift the additive model means so that they are the actual cell means. The larger this adjustment, the larger the interaction effect. In other words, the more that effect of one factor depends upon the levels of the other. 

Note that you do not *have* to include an interaction. A typical approach in Psychology is always to specify a *full factorial* model containing every main effect and every possible interaction. In general, this is probably the approach you should take, otherwise you make strong assumptions that all interactions are 0. However, there is nothing that says you *have* to do this. You have the flexibility to only include the effects that you want in the model. You just have to be aware of the consequences of doing so. 

$$
SS_{\text{model}} = SS_{A} + SS_{B} + SS_{AB}
$$

### Cell Means and Marginal Means


```{admonition} Interaction notation
:class: tip
There are different conventions for writing an interaction into a model. Some authors like to add an additional symbol to denote the interaction, leaving the subscripts to imply that the term is an interaction rather than a main effect. For instance

$$
y_{ijk} = \mu +\alpha_{i} + \beta_{j} + \gamma_{ij} + \epsilon_{ijk},
$$

where $\gamma_{ij}$ is the interaction. Personally, we like to use the following notation as it makes the interaction terms much more explicit, particularly in terms of which effects the interaction corresponds to

$$
y_{ijk} = \mu +\alpha_{i} + \beta_{j} + (\alpha\beta)_{ij} + \epsilon_{ijk}.
$$

For a model containing multiple interactions, this helps to make the meaning of the terms clearer, rather than somewhat hiding it in the subscripts. For instance, a 3-way full factorial ANOVA would be

$$
y_{ijkl} = \mu +\alpha_{i} + \beta_{j} + \gamma_{k} + (\alpha\beta)_{ij} + (\beta\gamma)_{jk} + (\alpha\gamma)_{ik} + (\alpha\beta\gamma)_{ijk} + \epsilon_{ijkl},
$$

where each main effect, 2-way interaction and 3-way interaction is clearly denoted.
```


### Interactions as Multiplicative Effects